In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setup notebook
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import LabelEncoder

data_path = "/content/drive/MyDrive/Colab Notebooks/time_series_data_analysis/store-sales-time-series-forecasting"


In [ ]:
# Set Matplotlib defaults
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

# Data cleanning and pre-processing

In [143]:



# load all the datasets
print("\ndataframe - data_holiday")
data_holiday = pd.read_csv(data_path + "/holidays_events.csv", parse_dates=["date"])
data_holiday.rename(columns={"type": "holiday_type"}, inplace=True)
data_holiday = data_holiday.drop_duplicates(subset=["date"], keep="last")
data_holiday["pay_day"] = (data_holiday['date'].dt.day == 15) | (data_holiday['date'].dt.is_month_end)

data_holiday['is_holiday'] = 1
data_holiday = data_holiday.set_index(['date'])
display(data_holiday.head())
display(data_holiday.shape)
display(data_holiday.dtypes)


print("\ndataframe - stores")
stores = pd.read_csv(data_path + "/stores.csv", dtype={
        'store_nbr': 'category',
        'type': 'category',
        'cluster': 'int'})
stores.rename(columns={"type": "store_type"}, inplace=True)
display(stores.head())
display(stores.shape)
display(stores.dtypes)

print("\ndataframe - oil original dateset")
oil_ori = pd.read_csv(data_path + "/oil.csv", parse_dates=["date"], dtype={
        'dcoilwtico': 'float32'})
display("with some missing value:", oil_ori.shape)

range_date = pd.date_range(start ='01/01/2013', end ='31/08/2017', freq ='D')
oil = pd.DataFrame(range_date, columns=['date'])
oil = oil.merge(oil_ori, on=["date"], how="left")
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='polynomial', order=2)
oil = oil[oil['date'] > "2015-12-31"]
oil.set_index("date", inplace=True)
oil = oil.sort_index()
display("After interpolation:")
display(oil.head())
display(oil.shape)
display(oil.dtypes)

print("\ndataframe - train_data")
train_data = pd.read_csv(data_path + "/train.csv", parse_dates=["date"], dtype={
        'store_nbr': 'category',
        'family': 'category',})
train_data = train_data[train_data['date'] > "2015-12-31"]
display(train_data.head())
display(train_data.shape)
display(train_data.dtypes)

print("\ndataframe - test_data")
test_data = pd.read_csv(data_path + '/test.csv',
        dtype={
        'store_nbr': 'category',
        'family': 'category'},parse_dates=['date'])
display(test_data.head())
display(test_data.shape)
display(test_data.dtypes)



dataframe - data_holiday


,holiday_type,locale,locale_name,description,transferred,pay_day,is_holiday
date,,,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,False,1
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,False,1
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,False,1
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,False,1
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,False,1


(312, 7)

holiday_type    object
locale          object
locale_name     object
description     object
transferred       bool
pay_day           bool
is_holiday       int64
dtype: object


dataframe - stores


...

id                      int64
date           datetime64[ns]
store_nbr            category
family               category
sales                 float64
onpromotion             int64
dtype: object


dataframe - test_data


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


(28512, 5)

id                      int64
date           datetime64[ns]
store_nbr            category
family               category
onpromotion             int64
dtype: object

In [ ]:
# not call this function
# def preprocess_data(df, field_mapping):
#     df["pay_day"] = (df['date'].dt.day == 15) | (df['date'].dt.is_month_end)
#     df["pay_day"] = df["pay_day"].apply(change_to_integer)
#     df = pd.merge(df, data_holiday, on =["date"], how="left", sort=True)
#     df = pd.merge(df, stores, on=["store_nbr"], how="left")
#     df = pd.merge(df, oil, on=["date"], how="left", sort = False)
#     df = df.drop_duplicates(subset=['date', 'store_nbr', 'family'])

#     df['holiday_type'].fillna(0, inplace=True)
#     df['locale'].fillna(0, inplace = True)
#     df["transferred"] = df["transferred"].apply(change_to_integer)
#     df.set_index(['id'], inplace = True)
#     # df.set_index(['date', 'store_nbr'], inplace=True)
#     df = df.sort_index()

#     for k, v in field_mapping.items():
#         df = df.replace({k: v[0]})
#     return df

# def all_category_mapping(holiday_df, store_df, train_df):
#     category_mappings = {}

#     family_names = list(train_df['family'].unique())
#     family_name_to_code = {family: i for i, family in enumerate(family_names)}
#     family_code_to_name = {i: family for i, family in enumerate(family_names)}

#     holiday_types = list(holiday_df['holiday_type'].unique())
#     holiday_type_to_code = {holidaytype: i+1 for i, holidaytype in enumerate(holiday_types)}
#     holiday_code_to_type = {i+1: holidaytype for i, holidaytype in enumerate(holiday_types)}

#     locale_types = list(holiday_df['locale'].unique())
#     locale_type_to_code = {localetype: i+1 for i, localetype in enumerate(locale_types)}
#     code_to_locale_type = {i+1: localetype for i, localetype in enumerate(locale_types)}

#     cities = list(store_df['city'].unique())
#     city_to_code = {city: i for i, city in enumerate(cities)}
#     code_to_city = {i: city for i, city in enumerate(cities)}

#     states = list(store_df['state'].unique())
#     state_to_code = {state: i for i, state in enumerate(states)}
#     code_to_state = {i: state for i, state in enumerate(states)}

#     store_types = list(store_df['store_type'].unique())
#     store_type_to_code = {store_type: i for i, store_type in enumerate(store_types)}
#     code_to_store_type = {i: store_type for i, store_type in enumerate(store_types)}

#     category_mappings= {"family": [family_name_to_code, family_code_to_name],
#                       "holiday_type": [holiday_type_to_code, holiday_code_to_type],
#                       "locale": [locale_type_to_code, code_to_locale_type],
#                       "city": [city_to_code, code_to_city],
#                       "state": [state_to_code, code_to_state],
#                       "store_type": [store_type_to_code, code_to_store_type]}
#     return category_mappings

# field_mapping = all_category_mapping(data_holiday, stores, train_data)

# train_df = preprocess_data(train_data, field_mapping)
# test_df = preprocess_data(test_data, field_mapping)

display(train_data.head())
display(test_data.head())


,id,date,store_nbr,family,sales,onpromotion
1945944,1945944,2016-01-01,1,AUTOMOTIVE,0.0,0
1945945,1945945,2016-01-01,1,BABY CARE,0.0,0
1945946,1945946,2016-01-01,1,BEAUTY,0.0,0
1945947,1945947,2016-01-01,1,BEVERAGES,0.0,0
1945948,1945948,2016-01-01,1,BOOKS,0.0,0


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [ ]:
# train_df.to_csv(data_path + "/pre_processed_train.csv")
# test_df.to_csv(data_path + "/pre_processed_test.csv")
# display(train_df.info())
# display(test_df.info())

# Model 1 - base model

In [161]:
from sklearn.preprocessing import StandardScaler

# pre-processing data_holiday dataframe
def change_to_integer(x):
    if x is True:
        return 1
    else:
        return 0

range_date = pd.date_range(start ='01/01/2016', end ='31/08/2017', freq ='D')
date_features = pd.DataFrame(range_date, columns=['date'])
date_features['pay_day'] = (date_features['date'].dt.day == 15) | (date_features['date'].dt.is_month_end)
date_features['pay_day'] = date_features['pay_day'].apply(change_to_integer)
date_features = date_features.merge(data_holiday[['is_holiday']], on=["date"], how="left")
date_features.drop_duplicates("date", keep="first", inplace=True)
date_features['is_holiday'] = date_features['is_holiday'].fillna(value=0).astype("int32")
date_features.set_index("date", inplace=True)
cols = date_features.columns

oil_data = oil.loc["2016-01-01":"2017-08-31"]
scaler = StandardScaler()
oil_scaled = scaler.fit_transform(oil_data)
oil_data['dcoilwtico'] = oil_scaled
date_features = pd.concat([date_features, oil_data], ignore_index = True, axis = 1)
# date_features = date_features.index.to_period("D")
date_features.columns = list(cols) + list(oil_data.columns)

train_date_features = date_features.loc[date_features.index < "2017-08-16"]
test_date_features = date_features[date_features.index > "2017-08-15"]
_
display(train_date_features)
display(test_date_features)

,pay_day,is_holiday,dcoilwtico
date,,,
2016-01-01,0,1,-1.270777
2016-01-02,0,0,-1.271919
2016-01-03,0,0,-1.325018
2016-01-04,0,0,-1.401163
2016-01-05,0,0,-1.534145
...,...,...,...
2017-08-11,0,1,0.498573
2017-08-12,0,0,0.478658
2017-08-13,0,0,0.360760


,pay_day,is_holiday,dcoilwtico
date,,,
2017-08-16,0,0,0.180367
2017-08-17,0,0,0.223111
2017-08-18,0,0,0.463744
2017-08-19,0,0,0.515369
2017-08-20,0,0,0.358813
2017-08-21,0,0,0.273770
2017-08-22,0,0,0.314932
2017-08-23,0,0,0.441581
2017-08-24,0,1,0.250024


In [168]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt

# data_path = "/content/drive/MyDrive/Colab Notebooks/time_series_data_analysis/store-sales-time-series-forecasting"
# train_df = pd.read_csv(data_path + "/pre_processed_train.csv", parse_dates=["date"])
# test_df = pd.read_csv(data_path + "/pre_processed_test.csv", parse_dates=["date"])

# train_df['date'] = train_df['date'].dt.to_period(freq = "D")
# test_df['date'] = test_df['date'].dt.to_period(freq = "D")

# according to the final generated output, choose the index and target column, and later we will unstack them to one row
y_ground = train_data[['date', 'store_nbr', 'family', 'sales']]
y_ground = y_ground.reset_index(drop=True)
y_train = y_ground.copy()
y_ground = y_ground.set_index(['date', 'store_nbr', 'family']).sort_index()

y_train['date'] = y_train['date'].dt.to_period('D')
y_train = y_train.set_index(['date', 'store_nbr', 'family']).sort_index()
y_train = y_train.unstack(['store_nbr', 'family'])

# print("y_train:")
# display(y_train)

# Create training data
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(index=y_train.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X_train = dp.in_sample()
X_test = dp.out_of_sample(steps=16)

# add NewYear, is_holiday, pay_day, oilPrice columns
X_train['NewYear'] = (X_train.index.dayofyear == 1)
cols = X_train.columns
X_train.index = X_train.index.astype('datetime64[ns]')
X_train = pd.concat([X_train, train_date_features], axis = 1)
X_train.columns = list(cols) + list(date_features.columns)
X_train.dropna(inplace=True)
print("X_train data:")
display(X_train)

# add NewYear, is_holiday, pay_day, oilPrice columns
X_test['NewYear'] = (X_test.index.dayofyear == 1)
cols = X_test.columns
X_test.index = X_test.index.astype('datetime64[ns]')
X_test = pd.concat([X_test, test_date_features], axis = 1)
X_test.columns = list(cols) + list(test_date_features.columns)
X_test.dropna(inplace=True)
print("X_test data:")
display(X_test)




y_train:


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2016-01-01        0.0       0.0    0.0       0.0   0.0      0.00000   
2016-01-02        7.0       0.0    0.0    1856.0   0.0    259.81100   
2016-01-03        1.0       0.0    2.0    1048.0   0.0    138.27500   
2016-01-04        8.0       0.0    2.0    3005.0   0.0    552.17200   
2016-01-05        7.0       0.0    4.0    2374.0   0.0    461.73800   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0    145.60700   
2017-08-12        6.0       0.0    3.0    1659.0   0.0    243.22000   
2017-08-13        1.0       0.0    1.0     803.0   0.0    136.67900   
2017-08-14        1.0       0.0    6.0    2201.0   0.0    346.03800   
2017-08-15        4.0       0.0    4.0    1942.0   0.0    329.54102   

                                                 ...                     \
store_nbr                                        ...         9            
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES    MEATS   
date                                             ...                      
2016-01-01         0.0      0.0    0.0    0.000  ...       0.0    0.000   
2016-01-02         5.0    526.0  627.0  115.960  ...      17.0  488.098   
2016-01-03         5.0    293.0  382.0   59.737  ...       6.0  595.221   
2016-01-04         8.0    938.0  983.0  144.129  ...       0.0  487.641   
2016-01-05        19.0    846.0  932.0  136.606  ...       1.0  390.266   
...                ...      ...    ...      ...  ...       ...      ...   
2017-08-11         4.0    341.0  343.0   64.302  ...       5.0  309.245   
2017-08-12         3.0    351.0  526.0   99.488  ...       2.0  260.298   
2017-08-13         1.0    169.0  266.0   47.770  ...       3.0  327.206   
2017-08-14         4.0    571.0  699.0  154.578  ...      12.0  330.975   
2017-08-15        21.0    703.0  602.0  116.402  ...      11.0  449.228   

                                                                          \
store_nbr                                                                  
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS    POULTRY   
date                                                                       
2016-01-01           0.0          0.0                     0.0    0.00000   
2016-01-02         851.0          5.0                    13.0  643.12897   
2016-01-03        1084.0         10.0                    19.0  748.38600   
2016-01-04         715.0          4.0                    13.0  487.42700   
2016-01-05         630.0          9.0                     7.0  432.36000   
...                  ...          ...                     ...        ...   
2017-08-11         373.0         11.0                     2.0  525.22400   
2017-08-12         400.0          7.0                    10.0  383.38700   
2017-08-13         510.0          2.0                     9.0  412.45800   
2017-08-14         445.0          2.0                    14.0  283.42900   
2017-08-15         522.0          6.0                     6.0  438.13300   

                                                                           
store_nbr                                                                  
family     PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                                                       
2016-01-01        0.00000     0.000                        0.0   0.000000  
2016-01-02      113.00000  2029.379                        4.0  24.146000  
2016-01-03      143.02200  3015.563                       13.0  21.277000  
2016-01-04       70.02800  1588.439                        6.0   7.000000  
2016-01-05       90.45000  2992.434                        3.0  23.200000  

X_train data:


,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",NewYear,pay_day,is_holiday,dcoilwtico
date,,,,,,,,,,,,,,,,,,,,
2016-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,True,0,1,-1.270777
2016-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967,False,0,0,-1.271919
2016-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649,False,0,0,-1.325018
2016-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758,False,0,0,-1.401163
2016-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869,False,0,0,-1.534145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,588.0,0.0,0.0,0.0,0.0,0.0,1.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653,False,0,1,0.498573
2017-08-12,1.0,589.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347,False,0,0,0.478658
2017-08-13,1.0,590.0,1.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139,False,0,0,0.360760


X_test data:


,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",NewYear,pay_day,is_holiday,dcoilwtico
2017-08-16,1.0,593.0,0.0,0.0,0.0,1.0,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,False,0,0,0.180367
2017-08-17,1.0,594.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.101168,-0.994869,0.201299,0.979530,-0.299363,-0.954139,0.394356,0.918958,False,0,0,0.223111
2017-08-18,1.0,595.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.299363,-0.954139,0.571268,0.820763,-0.790776,-0.612106,0.937752,0.347305,False,0,0,0.463744
2017-08-19,1.0,596.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.485302,-0.874347,0.848644,0.528964,-0.998717,-0.050649,0.897805,-0.440394,False,0,0,0.515369
2017-08-20,1.0,597.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.651372,-0.758758,0.988468,0.151428,-0.848644,0.528964,0.299363,-0.954139,False,0,0,0.358813
2017-08-21,1.0,598.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.790776,-0.612106,0.968077,-0.250653,-0.394356,0.918958,-0.485302,-0.874347,False,0,0,0.273770
2017-08-22,1.0,599.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.897805,-0.440394,0.790776,-0.612106,0.201299,0.979530,-0.968077,-0.250653,False,0,0,0.314932
2017-08-23,1.0,600.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.968077,-0.250653,0.485302,-0.874347,0.724793,0.688967,-0.848644,0.528964,False,0,0,0.441581
2017-08-24,1.0,601.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.998717,-0.050649,0.101168,-0.994869,0.988468,0.151428,-0.201299,0.979530,False,0,1,0.250024
2017-08-25,1.0,602.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.988468,0.151428,-0.299363,-0.954139,0.897805,-0.440394,0.571268,0.820763,False,0,0,0.314932


In [170]:
print("\nafter running DeterministicProcess, dp.in_sample, X shape: {}".format(X_train.shape))
display(X_train)

basic_model = LinearRegression(fit_intercept=True)      ###True or False?
basic_model.fit(X_train, y_train)
y_train_pred = pd.DataFrame(basic_model.predict(X_train), index=X_train.index, columns=y_train.columns)
print("\ngenerated y_train_pred shape:", y_train_pred.shape)
display(y_train_pred)

y_train_pred_1 = y_train_pred.stack(['store_nbr', 'family'])
y_train_residual  = y_ground - y_train_pred_1

print("\nresidual sales value after baisc model prediction:")

# Create features for test set

X_test['NewYear'] = (X_test.index.dayofyear == 1)
y_test_pred_1 = pd.DataFrame(basic_model.predict(X_test), index=X_test.index, columns=y_train.columns)
y_test_pred_1 = y_test_pred_1.stack(['store_nbr', 'family'])
print("\nsales prediction for forcast period:")

#######
# output for futher use:
y_train_pred_1
y_test_pred_1
y_train_residual


after running DeterministicProcess, dp.in_sample, X shape: (592, 20)


,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)","sin(4,freq=M)","cos(4,freq=M)",NewYear,pay_day,is_holiday,dcoilwtico
date,,,,,,,,,,,,,,,,,,,,
2016-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,True,0,1,-1.270777
2016-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967,False,0,0,-1.271919
2016-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649,False,0,0,-1.325018
2016-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758,False,0,0,-1.401163
2016-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869,False,0,0,-1.534145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,588.0,0.0,0.0,0.0,0.0,0.0,1.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653,False,0,1,0.498573
2017-08-12,1.0,589.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347,False,0,0,0.478658
2017-08-13,1.0,590.0,1.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139,False,0,0,0.360760



generated y_train_pred shape: (592, 1782)


sales                                                          \
store_nbr           1                                                           
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2016-01-01   0.700610       0.0 -0.022970   173.058333 -0.297072    53.741025   
2016-01-02   4.860082       0.0  2.232305  1808.794073 -0.042961   345.870884   
2016-01-03   3.692229       0.0  1.927005  1483.031039 -0.257762   281.196874   
2016-01-04   5.349923       0.0  2.962518  2232.755989  0.030386   440.334419   
2016-01-05   5.691015       0.0  2.623337  2179.240130 -0.056765   437.761646   
...               ...       ...       ...          ...       ...          ...   
2017-08-11   4.160753       0.0  3.562423  2122.921972  0.562389   331.604850   
2017-08-12   3.915476       0.0  3.829482  2417.431381  0.678617   385.039810   
2017-08-13   3.353642       0.0  2.918255  1920.066982  0.733514   281.481168   
2017-08-14   2.256174       0.0  2.752945  1641.162099  0.581481   220.333617   
2017-08-15   4.352778       0.0  3.635561  2227.372124  0.780588   340.568758   

                                                            ...            \
store_nbr                                                   ...         9   
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2016-01-01    4.046654  112.417953   86.082026   16.691152  ... -1.269667   
2016-01-02   10.895095  559.093842  692.027774  115.599609  ...  5.522786   
2016-01-03    7.336243  490.324735  560.042311   91.615009  ...  4.727410   
2016-01-04   13.884951  802.539414  797.167813  138.194086  ...  2.263668   
2016-01-05   15.513800  830.232232  805.896584  130.557882  ...  1.913876   
...                ...         ...         ...         ...  ...       ...   
2017-08-11   19.083862  636.417712  658.537108  133.883434  ...  3.957365   
2017-08-12   20.282262  671.974716  772.970859  150.991155  ...  3.622078   
2017-08-13   11.718573  457.453619  605.677648  111.525972  ...  5.706574   
2017-08-14    8.061759  407.510657  503.427275   95.984376  ...  5.115724   
2017-08-15   13.352309  677.686166  673.655679  134.821696  ...  4.208220   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2016-01-01  -41.965343    -79.435909    -1.914455               -3.705506   
2016-01-02  540.143452    890.418619     8.943331               15.218868   
2016-01-03  532.758571    869.705629     8.927258               13.600759   
2016-01-04  438.027186    672.556189     5.363751                8.827975   
2016-01-05  394.849238    578.115851     4.679531                7.092727   
...                ...           ...          ...                     ...   
2017-08-11  399.529470    370.387668     7.183379               10.234580   
2017-08-12  313.930737    378.639004     7.336123               10.916021   
2017-08-13  394.100751    530.641266     8.867085               14.761647   
2017-08-14  402.148679    516.217405     8.935200               13.450917   
2017-08-15  340.970168    433.085124     8.384102               11.786471   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2016-01-01  -37.818295     -21.871379  -273.859306                  -7.351497   
2016-01-02  734.964084     145.405507  2226.310956                  10.430593   
20


residual sales value after baisc model prediction:


sales
date       store_nbr family                                
2016-01-01 1         AUTOMOTIVE                   -0.700610
                     BABY CARE                     0.000000
                     BEAUTY                        0.022970
                     BEVERAGES                  -173.058333
                     BOOKS                         0.297072
...                                                     ...
2017-08-15 9         POULTRY                      51.401861
                     PREPARED FOODS               36.074243
                     PRODUCE                     595.598502
                     SCHOOL AND OFFICE SUPPLIES   99.969488
                     SEAFOOD                      -0.711922

[1054944 rows x 1 columns]


sales prediction for forcast period:


sales
           store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                     4.560979
                     BABY CARE                      0.000000
                     BEAUTY                         3.724081
                     BEVERAGES                   2464.833810
                     BOOKS                          0.738666
...                                                      ...
2017-08-31 9         POULTRY                      363.609105
                     PREPARED FOODS               124.431254
                     PRODUCE                     1448.045349
                     SCHOOL AND OFFICE SUPPLIES    21.316180
                     SEAFOOD                       16.631837

[28512 rows x 1 columns]

sales
date       store_nbr family                                
2016-01-01 1         AUTOMOTIVE                   -0.700610
                     BABY CARE                     0.000000
                     BEAUTY                        0.022970
                     BEVERAGES                  -173.058333
                     BOOKS                         0.297072
...                                                     ...
2017-08-15 9         POULTRY                      51.401861
                     PREPARED FOODS               36.074243
                     PRODUCE                     595.598502
                     SCHOOL AND OFFICE SUPPLIES   99.969488
                     SEAFOOD                      -0.711922

[1054944 rows x 1 columns]

In [171]:
# export the test data for submission
submit = pd.read_csv(data_path + "/sample_submission.csv", dtype={
        'id': 'object'})
submit['sales'] =  y_test_pred_1['sales'].reset_index(drop=True).to_list()
submit.to_csv(data_path + "/my_submission_claudia.csv", index = False)
display(submit)

,id,sales
0,3000888,4.560979
1,3000889,0.000000
2,3000890,3.724081
3,3000891,2464.833810
4,3000892,0.738666
...,...,...
28507,3029395,363.609105
28508,3029396,124.431254
28509,3029397,1448.045349
28510,3029398,21.316180


# Model 2 - Two boosted models are used for different family products, but they are both based on the residuals of Model 1

### Further processing the data to get used by the two sub models

In [ ]:

X_train = train_df.set_index(['date', 'store_nbr', 'family'])
X_train.pop("id")
X_train.pop("sales")
column_names = list(X_train.columns) +  list(y_train_residual.columns)
X_train = pd.concat([X_train, y_train_residual], ignore_index = True, axis = 1)
X_train.columns = column_names

X_test = test_df.set_index(['date', 'store_nbr', 'family'])
X_test.pop("id")

display(X_train.head())
display(X_test.head())

onpromotion  pay_day  holiday_type  locale  \
date       store_nbr family                                               
2016-01-01 1         0                 0        0             1       3   
                     1                 0        0             1       3   
                     2                 0        0             1       3   
                     3                 0        0             1       3   
                     4                 0        0             1       3   

                             transferred  city  state  store_type  cluster  \
date       store_nbr family                                                  
2016-01-01 1         0                 0     0      0           0       13   
                     1                 0     0      0           0       13   
                     2                 0     0      0           0       13   
                     3                 0     0      0           0       13   
                     4                 0     0      0           0       13   

                             dcoilwtico        sales  
date       store_nbr family                           
2016-01-01 1         0        37.633606    -5.165513  
                     1        37.633606     0.000000  
                     2        37.633606    -2.874160  
                     3        37.633606 -2158.723680  
                     4        37.633606     0.060250

onpromotion  pay_day  holiday_type  locale  \
date       store_nbr family                                               
2017-08-16 1         0                 0        0             0       0   
                     1                 0        0             0       0   
                     2                 2        0             0       0   
                     3                20        0             0       0   
                     4                 0        0             0       0   

                             transferred  city  state  store_type  cluster  \
date       store_nbr family                                                  
2017-08-16 1         0                 0     0      0           0       13   
                     1                 0     0      0           0       13   
                     2                 0     0      0           0       13   
                     3                 0     0      0           0       13   
                     4                 0     0      0           0       13   

                             dcoilwtico  
date       store_nbr family              
2017-08-16 1         0             46.8  
                     1             46.8  
                     2             46.8  
                     3             46.8  
                     4             46.8

In [ ]:
biweekly_columns =['BEVERAGES', 'BREAD/BAKERY', 'CLEANING', 'DAIRY', 'EGGS', "FROZEN FOODS", 'GROCERY I', 'HOME APPLIANCES', 'HOME CARE', 'LINGERIE','MEATS', 'POULTRY', 'PRODUCE']
holiday_columns = ['CELEBRATION','LAWN AND GARDEN','LIQUOR,WINE,BEER', 'PERSONAL CARE', 'PREPARED FOODS', 'SEAFOOD']
ready_columns= ['AUTOMOTIVE', "BABY CARE", 'BEAUTY', "BOOKS", "GROCERY II", "HARDWARE", "HOME AND KITCHEN I", "HOME AND KITCHEN II", "LADIESWEAR", 'MAGAZINES', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'SCHOOL AND OFFICE SUPPLIES']

# fieldmapping comes from part 1
biweekly_code = []
for column_name in biweekly_columns:
    biweekly_code.append(field_mapping['family'][0][column_name])

holiday_code = []
for column_name in holiday_columns:
    holiday_code.append(field_mapping['family'][0][column_name])

# display(biweekly_code)
# display(holiday_code)


In [ ]:
import numpy as np

X_train_biweekly = X_train[np.in1d(X_train.index.get_level_values(2), biweekly_code)]
X_train_holiday =  X_train[np.in1d(X_train.index.get_level_values(2), holiday_code)]
display(X_train_biweekly)
display(X_train_holiday)

y_train_biweekly = X_train_biweekly.pop('sales')
y_train_holiday = X_train_holiday.pop("sales")

X_test_biweekly = X_test[np.in1d(X_test.index.get_level_values(2), biweekly_code)]
display(X_train_biweekly)

X_test_holiday =  X_test[np.in1d(X_test.index.get_level_values(2), holiday_code)]
display(X_test_holiday)
display(X_test_biweekly)
display(X_test_holiday)


### Start the bi-model training

In [ ]:

# remove the columns un-related to payment day
for column in ['holiday_type', 'locale', 'transferred']:
    X_train_biweekly.pop(column)
for column in ['holiday_type', 'locale', 'transferred']:
    X_test_biweekly.pop(column)

from xgboost import XGBRegressor

# Fit and predict
model_2 = XGBRegressor()
model_2.fit(X_train_biweekly, y_train_biweekly)

y_train_pred_biweekly = pd.DataFrame(model_2.predict(X_train_biweekly), index=X_train_biweekly.index)
y_test_pred_biweekly = pd.DataFrame(model_2.predict(X_test_biweekly), index=X_test_biweekly.index)
y_test_pred_biweekly.columns = ['sales']

In [ ]:
# Fit and predict

X_train_holiday.pop("pay_day")
X_test_holiday.pop("pay_day")

model_3 = XGBRegressor()
model_3.fit(X_train_holiday, y_train_holiday)
y_train_pred_holiday = pd.DataFrame(model_3.predict(X_train_holiday), index=X_train_holiday.index)
y_train_pred_holiday.columns = ['sales']

y_test_pred_holiday = pd.DataFrame(model_3.predict(X_test_holiday), index=X_test_holiday.index)
y_test_pred_holiday.columns = ['sales']

### Concatenate prediction results from both models

In [ ]:
y_train_total = pd.concat([y_ground, y_train_pred_1, y_train_pred_biweekly, y_train_pred_holiday], axis = 1)
y_train_total.columns = ['ground', 'model_1', "model_2", "model_3"]
y_train_total.fillna(0.0, inplace = True)
y_train_total['sales_forcast'] = y_train_total['model_1'] + y_train_total['model_2'] + y_train_total['model_3']
display(y_train_total)

y_test_total = pd.concat([y_test_pred_1, y_test_pred_biweekly, y_test_pred_holiday], axis = 1)
y_test_total.columns = ['model_1', "model_2", "model_3"]
y_test_total.fillna(0.0, inplace = True)
y_test_total['sales_forcast'] = y_test_total['model_1'] + y_test_total['model_2'] + y_test_total['model_3']
display(y_test_total)

submit = pd.read_csv(data_path + "/sample_submission.csv", dtype={
        'id': 'object'})
submit['sales'] =  y_test_total['sales_forcast'].reset_index(drop=True).to_list()
submit.to_csv(data_path + "/my_submission_claudia.csv", index = False)
display(submit)

model_1     model_2   model_3  sales_forcast
date       store_nbr family                                                  
2017-08-16 1         0          4.366016    0.000000  0.000000       4.366016
                     1          0.000000    0.000000  0.000000       0.000000
                     2          3.801801    0.000000  0.000000       3.801801
                     3       2437.876680 -101.422096  0.000000    2336.454584
                     4          0.888709    0.000000  0.000000       0.888709
...                                  ...         ...       ...            ...
2017-08-31 54        28        65.167308   -5.911600  0.000000      59.255708
                     29        79.528900    0.000000 -1.891784      77.637116
                     30       556.115684  -37.974304  0.000000     518.141379
                     31         2.852812    0.000000  0.000000       2.852812
                     32         2.504650    0.000000 -1.891784       0.612866

[28512 rows x 4 columns]

In [ ]:
###################################
#  refer to this post https://www.kaggle.com/code/ryanholbrook/hybrid-models
# https://www.kaggle.com/code/claudiayao/time-series-as-features/edit

# all the above family products's base model has been built up in the other notebook, it need to build another model based on the residul value

In [ ]:
submit = pd.read_csv(data_path + "/sample_submission.csv", dtype={
        'id': 'object'})
submit['sales'] =  y_test_pred_1['sales'].reset_index(drop=True).to_list()
submit.to_csv(data_path + "/my_submission_claudia.csv", index = False)
display(submit)

,id,sales
0,3000888,4.346402
1,3000889,0.000000
2,3000890,3.789654
3,3000891,2429.384594
4,3000892,0.888033
...,...,...
28507,3029395,66.591154
28508,3029396,80.859348
28509,3029397,569.776679
28510,3029398,2.900855
